# 📩 Self Sufficient Kaggle Submission

### Important Considerations for This Approach

This simplified approach may not suit your needs if your solution depends on an external model file. In that case, you should consider using the alternative approach (which involves importing and using an external package).

✅ Succedded as an ARC Prize 2024 submission (scored 0.0 because no solution was implemented).

### 🚨 Key Reminder 🚨

Make sure to **implement your solution** within the `predict()` function provided in this notebook. This is crucial for your model to function correctly during evaluation.

### Differences from the Full Framework Version

This notebook contains a minimal version of the framework, designed for streamlined usage in the Kaggle environment, with a few adjustments:
- **Docstrings Removed**: To save space, detailed docstrings for classes, methods, and functions have been omitted here. You can find the full documentation within the base framework.
- **No Graphics**: The `Graphics` class, which is responsible for plotting challenges, has been excluded because Kaggle's environment lacks an interactive interface. Consequently, the `Solver` class has also been shortened, omitting any plotting functionalities.

### Efficient Handling of Evaluation

This code implements several optimizations to ensure smooth handling during evaluation:
- **Fallback for Failed Predictions**: If your prediction fails, the code automatically fills the submission file with a random answer. This ensures that the file format remains intact and avoids breaking the submission.
- **Challenge Ordering**: Challenges are processed in ascending order of size, optimizing the number of completed challenges in case the evaluation runs out of time.
- **Incremental Submission Writing**: After each prediction, the result is immediately added to the submission file. Unlike other approaches that predict all solutions first and then write to the file at the end (which can disrupt evaluation), this method ensures each result is promptly stored.


In [1]:
import os
import json
import random
import numpy as np
from tqdm import tqdm

In [2]:
def shuffle_list(input_list):
    random.shuffle(input_list)
    return input_list


def load_json(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

def manhattan_distance(xa, ya, xb, yb):
    return abs(xa - xb) + abs(ya - yb)


def calculate_center(points):
    x_coordinates = [point[0] for point in points]
    y_coordinates = [point[1] for point in points]

    center_x = sum(x_coordinates) / len(x_coordinates)
    center_y = sum(y_coordinates) / len(y_coordinates)

    return (center_x, center_y)


def sort_challenges_by_size(challenges, ascending=True):
    def count_challenge_cells(challenge):
        return sum(
            extract_numbers(example['input']) + extract_numbers(example['output']) 
            for example in challenge['train']
        )

    def extract_numbers(list_of_lists):
        return sum(len(sublist) for sublist in list_of_lists)
    
    def check_ids(list1, list2):
        return sorted(list1) == sorted(list2)
    
    def sort_ids_by_numbers(ids, numbers, ascending=True):
        return [id for _, id in sorted(zip(numbers, ids), reverse=not ascending)]
        
    challenge_ids = list(challenges)
    numbers = [count_challenge_cells(challenges[_id]) for _id in challenge_ids]

    return sort_ids_by_numbers(challenge_ids, numbers, ascending=ascending)

In [3]:
class Solver:
    def __init__(self, prod=False) -> None:
        if prod:
            self.base_path = '/kaggle/input/arc-prize-2024/'
        else:
            self.base_path = '../data/challenges/'

        self.training_challenges = load_json(
            self.base_path + 'arc-agi_training_challenges.json')
        self.training_solutions = load_json(
            self.base_path + 'arc-agi_training_solutions.json')
        self.evaluation_challenges = load_json(
            self.base_path + 'arc-agi_evaluation_challenges.json')
        self.evaluation_solutions = load_json(
            self.base_path + 'arc-agi_evaluation_solutions.json')
        self.test_challenges = load_json(
            self.base_path + 'arc-agi_test_challenges.json')
        self.sample_submission = load_json(
            self.base_path + 'sample_submission.json')

    def predict(self, puzzle_inps_train, puzzle_outs_train, puzzle_inps_test, puzzle_outs_test=None):
        try:
            # ! Your board prediction solution goes here ! #
            raise NotImplementedError
        except Exception:
            answers = self.random_prediction(
                puzzle_outs_train, puzzle_inps_test)

        return answers

    def random_prediction(self, puzzle_outs_train, puzzle_inps_test):
        answers = []
        avg_shape = np.ceil(np.array([np.array(p.shape) for p in puzzle_outs_train]).mean(
            0)).astype(int)
        for _ in range(len(puzzle_inps_test)):
            answers.append(np.random.randint(0, 10, size=avg_shape))

        return answers

    def train(self):
        raise NotImplementedError

    def validate(self):
        raise NotImplementedError

    def test(self):
        raise NotImplementedError

    def get_challenge_board(self, challenge_id, challenges, solutions, io: str, board_type: str, board_idx):

        board = None
        if challenge_id in list(challenges):
            puzzle_inps_train, puzzle_outs_train, puzzle_inps_test, puzzle_outs_test = self.process_challenge(
                challenge_id, challenges, solutions)
            if io == 'input':
                if board_type == 'train':
                    if board_idx in range(0, len(puzzle_inps_train)):
                        board = puzzle_inps_train[board_idx]
                else:
                    if board_idx in range(0, len(puzzle_inps_test)):
                        board = puzzle_inps_test[board_idx]
            else:
                if board_type == 'train':
                    if board_idx in range(0, len(puzzle_outs_train)):
                        board = puzzle_outs_train[board_idx]
                else:
                    if board_idx in range(0, len(puzzle_outs_test)):
                        board = puzzle_outs_test[board_idx]
        return board

    def process_challenge(self, challenge_id, challenges, solutions=None):
        one_challenge = challenges[challenge_id]

        puzzle_inps_train = []
        puzzle_outs_train = []
        for puzzles in one_challenge['train']:
            puzzle_inps_train.append(np.array(puzzles['input']))
            puzzle_outs_train.append(np.array(puzzles['output']))

        puzzle_inps_test = []
        for puzzles in one_challenge['test']:
            puzzle_inps_test.append(np.array(puzzles['input']))

        if solutions != None:
            one_solution = solutions[challenge_id]
            puzzle_outs_test = []
            for puzzles in one_solution:
                puzzle_outs_test.append(np.array(puzzles))

            return puzzle_inps_train, puzzle_outs_train, puzzle_inps_test, puzzle_outs_test

        else:
            return puzzle_inps_train, puzzle_outs_train, puzzle_inps_test, None

In [4]:
# Initialize solver
anr = Solver(prod=True)

base_path = '/kaggle/input/arc-prize-2024/' # path may change in 2025 arc-prize
test_challenges = anr.test_challenges
submission_file_path = '/kaggle/working/submission.json'
sample_submission_file_path = base_path + 'sample_submission.json'

In [ ]:
sample_submission = load_json(sample_submission_file_path)

with open(submission_file_path, "w") as file:
        json.dump(sample_submission, file, indent=4)

In [6]:
with open(submission_file_path, "r+") as outfile:
    submission_data = json.load(outfile)
    
    ids_test = sort_challenges_by_size(test_challenges)

    for i, challenge_id in enumerate(tqdm(ids_test)):
        puzzle_ins_train, puzzle_outs_train, puzzle_ins_test, puzzle_outs_test = anr.process_challenge(challenge_id, test_challenges)
        
        attempt1 = anr.predict(puzzle_ins_train, puzzle_outs_train, puzzle_ins_test)
        attempt2 = anr.predict(puzzle_ins_train, puzzle_outs_train, puzzle_ins_test)
        
        result = []
        for j in range(len(attempt1)):
            result.append({
                'attempt_1': attempt1[j].tolist(),
                'attempt_2': attempt2[j].tolist()
            })
        
        submission_data[challenge_id] = result
        
        outfile.seek(0)
        json.dump(submission_data, outfile, indent=4)
        outfile.truncate()

100%|██████████| 100/100 [00:02<00:00, 43.01it/s]


### 💡 Save Yourself the Headache! 💡

I've made the mistakes so you don't have to! Now, you'll save tons of time working on ARC-AGI. 

If this notebook helped you avoid common pitfalls or sped up your progress, I'd love your support!

- **Follow me on Kaggle:** [Malo Le Mestre](https://www.kaggle.com/malolem)
- **Leave a ⭐ on the GitHub repo** [here](https://github.com/MaloLM/arc-agi-genesis) to show your appreciation and keep the project growing!
- **Upvote this notebook** on Kaggle if it saved you from banging your head against the wall!

Your feedback keeps me motivated and helps others avoid the same challenges. 

# Thank you! 🚀✨